## Setup//Tokens

In [ ]:
import os
import json
from fabrictestbed.slice_manager import SliceManager, Status

fabric_refresh_token = None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token = os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
    
print("Refresh token is: {}".format(fabric_refresh_token))

credentialmanager_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']

print(f"CM Host: {credentialmanager_host} Orchestrator Host: {orchestrator_host}")

#create slice manager
slice_manager = SliceManager(oc_host = orchestrator_host, cm_host = credentialmanager_host, refresh_token = fabric_refresh_token, project_name='all', scope='all')

#ssh
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key = myfile.read().strip()
  

#get id token
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
fabric_id_token=slice_manager.get_id_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("New Id Token: {}".format(fabric_id_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token
print()
print("Stroed new ID Token")
%store fabric_id_token
print()

## Setup//Tokens NEW VERSION

In [21]:
import json
import os
from fabrictestbed.slice_manager import SliceManager, Status

credentialmanager_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']

print(f"CM Host: {credentialmanager_host} Orchestrator Host: {orchestrator_host}")
print()

slice_manager = SliceManager(oc_host = orchestrator_host, cm_host = credentialmanager_host, project_name='all', scope='all')
slice_manager.initialize()

print("Token location: {}".format(slice_manager.token_location))

fabric_refresh_token = slice_manager.get_refresh_token()
fabric_id_token = slice_manager.get_id_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("New Id Token: {}".format(fabric_id_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token
print()
print("Stroed new ID Token")
%store fabric_id_token
print()

ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net

Token location: /home/fabric/.tokens.json

New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTEMLCME4GIMRYMYYGEOBRGIYDINLCGBRDIMTBMU2DMOJRGQ4TSZR7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRWG4YTCNJRGMYDKNRGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ

New Id Token: eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImI0MTUxNjcyMTExOTFlMmUwNWIyMmI1NGIxZDNiNzY2N2U3NjRhNzQ3NzIyMTg1ZTcyMmU1MmUxNDZmZTQzYWEifQ.eyJlbWFpbCI6Im5hbm5rYXRAYnUuZWR1IiwiY2VydF9zdWJqZWN0X2RuIjoiL0RDPW9yZy9EQz1jaWxvZ29uL0M9VVMvTz1Cb3N0b24gVW5pdmVyc2l0eS9DTj1OYW5uYSBLYXRyaW4gSGFubmVzZG90dGlyIEE0NjA5NjM1NiIsImlkcCI6Imh0dHBzOi8vc2hpYi5idS5lZHUvaWRwL3NoaWJib2xldGgiLCJpZHBfbmFtZSI6IkJvc3RvbiBVbml2ZXJzaXR5IiwiZXBwbiI6Im5hbm5rYXRAYnUuZWR1IiwiZXB0aWQiOiJodHRwczovL3NoaWIuYnUuZWR1L2lkcC9zaGliYm9sZXRoIWh0dHBzOi8vY2lsb2dvbi5vcmcvc2hpYmJvbGV0aCFwZW5PVjQzcXF5blR1Tjg5T2hqNVJqcnVQUTA9IiwiYWZmaWxpYXRpb24iOiJzdHVkZW50QGJ1LmVkdTttZW1iZXJAYnUuZWR1Iiwibm

In [22]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, LinkType, Layer

## Create Slice 1

### Experiment obj and node

In [47]:
experiment = ExperimentTopology()
slice1_name = "TestSlice1"

slice1_node = experiment.add_node(name='s1n1', site = 'UKY')

slice1_node_capacities = Capacities()
slice1_node_capacities.set_fields(core = 4, ram = 16, disk = 100)
slice1_node.set_properties(capacities = slice1_node_capacities, image_type='qcow2', image_ref='default_ubuntu_20')

#slice1_node2 = experiment.add_node(name='s1n2', site = 'UKY')
#slice1_node2_capacities = Capacities()
#slice1_node2_capacities.set_fields(core=2, ram=12, disk=20)
#slice1_node2.set_properties(capacities = slice1_node2_capacities, image_type='qcow2', image_ref='default_ubuntu_20')

### Configure slice

In [48]:
#serialize into slice graph object
slice1_graph = experiment.serialize()

#communicate with orchestrator for slice
status,reservations = slice_manager.create(slice_name = slice1_name, slice_graph = slice1_graph, ssh_key = ssh_key)

#get the slice id of the newly created slice and check status to see if it went through
print("Request {}".format(status))
print("Reservations: {}".format(reservations))

slice1_id = reservations[0].slice_id

Request Status.OK
Reservations: [{
    "graph_node_id": "0153fc7a-57c0-477e-9b15-a4f9669d564d",
    "lease_end": "2021-07-20 16:39:16",
    "pending_state": "None_",
    "reservation_id": "b31b7053-94d8-41e6-bb64-4016440e37dc",
    "reservation_state": "Unknown",
    "slice_id": "686a5c9b-0a55-4689-9150-234bb257336d"
}]


### Query slices

In [52]:
import time
status, slices_all = slice_manager.slices(state = 'All')
slices = list(filter(lambda x: x.slice_state == 'Configuring' or x.slice_state == 'StableOK', slices_all ))

print("Response Status {}".format(status))
print("Slices {}".format(slices))

Response Status Status.OK
Slices [{
    "graph_id": "2bdc90bd-68c5-49a0-b97e-0fa14f480ee0",
    "lease_end": "2021-07-20 16:39:16",
    "slice_id": "686a5c9b-0a55-4689-9150-234bb257336d",
    "slice_name": "TestSlice1",
    "slice_state": "StableOK"
}]


In [51]:
state = None
while state != 'StableOK':
    status, slice_status = slice_manager.slice_status(slice_id=slice1_id)
    state = slice_status.slice_state
    print("Response Status {}".format(status))
    print("Slice State: {}".format(slices[0].slice_state))
    time.sleep(20)

Response Status Status.OK
Slice State: Configuring
Response Status Status.OK
Slice State: Configuring


## Create Slice 2

### Experiment obj and node

In [58]:
experiment = ExperimentTopology()
slice2_name = "TestSlice2"

slice2_node = experiment.add_node(name='s2n1', site = 'LBNL')

slice2_node_capacities = Capacities()
slice2_node_capacities.set_fields(core=4, ram=20, disk=80)
slice2_node.set_properties(capacities = slice2_node_capacities, image_type='qcow2', image_ref='default_ubuntu_20')

### Configure slice

In [59]:
#serialize into slice graph object
slice2_graph = experiment.serialize()

#communicate with orchestrator for slice
status,reservations = slice_manager.create(slice_name = slice2_name, slice_graph = slice2_graph, ssh_key = ssh_key)

#get the slice id of the newly created slice and check status to see if it went through
print("Request {}".format(status))
print("Reservations: {}".format(reservations))

slice2_id = reservations[-1].slice_id

Request Status.OK
Reservations: [{
    "graph_node_id": "d45ebe78-e7f2-4195-9c96-f75cba47688f",
    "lease_end": "2021-07-20 16:44:11",
    "pending_state": "None_",
    "reservation_id": "96fbbec5-2960-4b9f-a38f-52dd3ede643d",
    "reservation_state": "Unknown",
    "slice_id": "47d88cef-f180-40eb-9ef1-68dfa8ba43eb"
}]


### Query slices

In [62]:
import time
status, slices_all = slice_manager.slices(state = 'All')
slices = list(filter(lambda x: x.slice_state == 'Configuring' or x.slice_state == 'StableOK', slices_all ))

print("Response Status {}".format(status))
print("Slices {}".format(slices))

Response Status Status.OK
Slices [{
    "graph_id": "a89babdf-048d-4b47-a23c-97f87342ca2e",
    "lease_end": "2021-07-20 16:44:11",
    "slice_id": "47d88cef-f180-40eb-9ef1-68dfa8ba43eb",
    "slice_name": "TestSlice2",
    "slice_state": "StableOK"
}, {
    "graph_id": "2bdc90bd-68c5-49a0-b97e-0fa14f480ee0",
    "lease_end": "2021-07-20 16:39:16",
    "slice_id": "686a5c9b-0a55-4689-9150-234bb257336d",
    "slice_name": "TestSlice1",
    "slice_state": "StableOK"
}]


In [61]:
state = None
while state != 'StableOK':
    status, slice_status = slice_manager.slice_status(slice_id=slice2_id)
    state = slice_status.slice_state
    print("Response Status {}".format(status))
    print("Slice State: {}".format(slices[0].slice_state))
    time.sleep(20)

Response Status Status.OK
Slice State: StableOK
Response Status Status.OK
Slice State: StableOK
Response Status Status.OK
Slice State: StableOK


## Display slices with map

In [63]:
status, slices = slice_manager.slices(state = 'Active')

print("Response Status {}".format(status))
print("Slices {}".format(slices))

Response Status Status.OK
Slices [{
    "graph_id": "a89babdf-048d-4b47-a23c-97f87342ca2e",
    "lease_end": "2021-07-20 16:44:11",
    "slice_id": "47d88cef-f180-40eb-9ef1-68dfa8ba43eb",
    "slice_name": "TestSlice2",
    "slice_state": "StableOK"
}, {
    "graph_id": "2bdc90bd-68c5-49a0-b97e-0fa14f480ee0",
    "lease_end": "2021-07-20 16:39:16",
    "slice_id": "686a5c9b-0a55-4689-9150-234bb257336d",
    "slice_name": "TestSlice1",
    "slice_state": "StableOK"
}]


In [64]:
site_dict = {'UKY': (38.03154264461649, -84.50343326019218), 'RENCI': (35.93996154497003, -79.01809264307425), 'LBNL': (38.94977311613871, -122.61289106281062)}

In [65]:
for slice in slices:
    status, nodes = slice_manager.slivers(slice_id = slice.slice_id)
    print(nodes[0].allocated_capacities)

{ core: 8 , ram: 32 G, disk: 100 G}
{ core: 4 , ram: 16 G, disk: 100 G}


In [66]:
import ipywidgets as widgets
from ipywidgets import Text, HTML, HBox, Layout, VBox
from ipyleaflet import (
    Map,
    Marker,
    TileLayer, ImageOverlay,
    Polyline, Polygon, Rectangle, Circle, CircleMarker,
    GeoJSON,
    DrawControl,
    WidgetControl,
    basemaps,
    Popup,
    Icon,
    MarkerCluster
)

slice_list = []
slice_info = {}

for slice in slices:
    status, nodes = slice_manager.slivers(slice_id = slice.slice_id)
    slice_list.append(slice.slice_name)
    slice_info[slice.slice_name] = nodes


#slice_list.append('Synthetic_slice')
#slice_info['Synthetic_slice'] = 'LBNL'

selection = widgets.RadioButtons(
    options = slice_list,
    description = 'My slices:',
    disabled=False
)


def display_slice(slice):
    slice_map = Map(basemap = basemaps.Esri.WorldStreetMap, center=(38.12480976137421, -95.7129), zoom = 3.8)
    title = [widgets.HTML(value = '<center><b>slice:</b> ' + slice + '</center>')]
   
    my_layout = Layout(width = '200px', overflow_y= 'auto', padding = "4px" )
    map_title = VBox(title , layout = my_layout )
    control = WidgetControl(widget = map_title, position='topright')
    slice_map.add_control(control)
    
    marker_list = []
    for node in slice_info[slice]:
        site = node.site
        b = HTML(value = '<b>' + node.name + '</b><br><b>location:</b> '+ site +'<br><b>capacities:</b> core: ' + str(node.allocated_capacities.core) + ', ram: ' + str(node.allocated_capacities.ram) +
                 ' G, disk: '+ str(node.allocated_capacities.disk) + ' G<br><b>components:</b><br><b>links:</b>', disabled=True) 
        html_info = HBox([b], layout=Layout(overflow_y='auto'))
        node = Marker(location = site_dict[site], draggable = False, title = site)
        node.popup = html_info 
        marker_list.append(node)
        
    my_cluster = MarkerCluster(markers = tuple(marker_list))
    slice_map.add_layer(node)
    display(slice_map)

#height='160px

widgets.interact(display_slice, slice = selection)


interactive(children=(RadioButtons(description='My slices:', options=('TestSlice2', 'TestSlice1'), value='Test…

<function __main__.display_slice(slice)>

## Delete slices

In [67]:
status, result = slice_manager.delete(slice_id = slice1_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

status, result = slice_manager.delete(slice_id = slice2_id)

print("Response Status {}".format(status))
print("Response received {}".format(result))

Response Status Status.OK
Response received None
Response Status Status.OK
Response received None
